# Austin Deep Learning Meetup
### Tuesday Oct 18, 2016
____

# Data Munging for Deep Learning
### Danny Mulligan, danny@dannymulligan.com

## This presentation is available on GitHub at
### https://github.com/dannymulligan/public/tree/master/ADLM

More than half the time you spend working on machine learning is actually spent preparing the data.  And yet most machine learning demos focus only on the machine learning, and ignore the data prep.  "Here's some data I prepared earlier".

In this presentation we'll focus on the data prep steps in Python.

- Getting the data
- Dealing with missing data
- Grouping, filtering, sorting
- Feature generation
- Normalizing, fuzzing, binning and other data transforms
- Splitting data into training & test sets
- Saving and loading data
- ...anything else you want to do to prepare your data

----

# What are the most important Python libraries for Data Munging?

## Python Standard Library
- Lots and lots of useful functions
- You need to have a good idea of what's in this library
- If you don't, spend an hour skimming the documentation
- Really, you should know what's in the standard library

### https://docs.python.org/3/library/

## Numpy
- High performance arrays
- Allows you to use highly optimized numerical libraries with almost zero effort

### https://docs.scipy.org/doc/numpy-1.11.0/reference/

## Pandas
- Numpy arrays with fancy indexing
- Lots of other data processing productivity tools

### http://pandas.pydata.org/pandas-docs/stable/

## Jupyter/IPython notebooks
- Great for data exploration, learning the libraries, experimenting with machine learning, etc
- **This** is a Jupyter notebook

### https://jupyter.readthedocs.io/en/latest/
____

# Warning:
### I wrote this notebook using Python 3.x but I never tested with Python 2.x.  If you have problems make sure you are using Python 3.x
### You MUST have Pandas & Numpy installed to run much of the code in this notebook.
#### I recommend installing Anaconda if you want these (plus a bunch of other very useful) libraries.
#### Download Anaconda from here: https://www.continuum.io/downloads

In [1]:
import datetime      # These imports are from the standard Python Library, they should always work
import json
import random
import sys
print("Python version", sys.version)

import numpy as np   # If this fails, you need to install Numpy
print("Numpy version", np.__version__)

import pandas as pd  # If this fails, you need to install Pandas
print("Pandas version", pd.__version__)

Python version 3.5.2 |Anaconda 4.0.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
Numpy version 1.11.0
Pandas version 0.18.0


# But wait, isn't Python too slow for processing large amounts of data?

#### It depends on what you're doing, but probably not.

#### If the heavy lifting happens *inside* a Python library, it will be just as fast as practically any other language.

In [2]:
# Make a list containing random numbers between 0.0 & 1.0
MyList = [random.random() for x in range(1000)]
MyList[:5]  # print the first 5 items in the list

[0.2903127187437302,
 0.579317589392338,
 0.512775914305855,
 0.2210105502871651,
 0.4681933373575832]

In [3]:
def MyMax(list):
    '''Find the largest number in a list'''
    Max = list[0]
    for x in list[1:]:
        if x > Max:
            Max = x
    return Max

In [4]:
%timeit MyMax(MyList)
# When I ran this, I got...
# 10000 loops, best of 3: 40.1 µs per loop

10000 loops, best of 3: 38.8 µs per loop


In [5]:
%timeit max(MyList)
# When I ran this, I got...
# 10000 loops, best of 3: 25.9 µs per loop

10000 loops, best of 3: 24.8 µs per loop


In [7]:
MyList2 = np.array(MyList)
%timeit np.max(MyList2)

The slowest run took 10.89 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.25 µs per loop


### When I ran the above experiments, I got these results

#### 37.1 µs = MyMax function
#### 26.5 µs = Python library max (1.4x faster)
#### 5.55 µs = Numpy library max (6.7x faster)

### Try the same thing with sorting

In [8]:
# From: https://stackoverflow.com/questions/18262306/quick-sort-with-python
# Written by https://stackoverflow.com/users/3011380/zangw
def qsort(arr): 
     if len(arr) <= 1:
          return arr
     else:
          return qsort([x for x in arr[1:] if x<arr[0]]) + \
                 [arr[0]] +                                \
                 qsort([x for x in arr[1:] if x>=arr[0]])

In [9]:
%timeit MyList3 = qsort(MyList)
MyList3 = qsort(MyList)
MyList3[:5]

100 loops, best of 3: 3.49 ms per loop


[0.0020763875951721467,
 0.003051551209218628,
 0.0043363095090050985,
 0.004670273228042587,
 0.004741018308700884]

In [10]:
%timeit MyList4 = MyList.copy(); MyList4.sort()
MyList4 = MyList.copy(); MyList4.sort()
MyList4[:5]

1000 loops, best of 3: 211 µs per loop


[0.0020763875951721467,
 0.003051551209218628,
 0.0043363095090050985,
 0.004670273228042587,
 0.004741018308700884]

In [11]:
MyList5 = np.array(MyList)
%timeit MyList6 = np.sort(MyList5)
MyList6 = np.sort(MyList5)
MyList6[:5]

The slowest run took 4.31 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 26.2 µs per loop


array([ 0.00207639,  0.00305155,  0.00433631,  0.00467027,  0.00474102])

### When I ran the above sorting experiments, I got these results

#### 3,720 µs = qsort function
#### 227 µs = Python library sort (16.4x faster)
#### 28.9 µs = Numpy library sort (128.7x faster)

## Conclusion: Python is plenty fast enough, if you are using optimized libraries for the heavy lifting

____

# Getting data

- Before you can process data, you've got to get it, and then get it into Python.
- Lots and lots of data is available on the Internet, so lets get something from there.

Here are example of some things I found with a Google search for "interesting datasets csv"

http://statweb.stanford.edu/~sabatti/data.html

https://www.comptroller.texas.gov/transparency/open-data/search-datasets/

https://catalog.data.gov/dataset?res_format=CSV
    
### Since we're focusing on the data munging, not the data, we're going to work with something simple like stock quotes.

#### Stock quotes are available from Yahoo at:
    http://ichart.finance.yahoo.com/table.csv?d=6&e=1&f=2009&g=d&a=7&b=19&c=2004&ignore=.csv&s=[SymbolName]

With a little examination, we can figure out how to build a URL to fetch any stock data we want at this link.

In [12]:
def MakeYahooStockURL(Ticker, Start, End):
    '''Make a URL to download stock data from Yahoo'''
    HeadURL = 'http://ichart.finance.yahoo.com/table.csv?'
    StartURL = '&a={}&b={}&c={}'.format(Start.month-1, Start.day, Start.year)
    EndURL   = '&d={}&e={}&f={}'.format(  End.month-1,   End.day,   End.year)
    TailURL = '&g=d&ignore=.csv&s={}'.format(Ticker)
    return HeadURL + StartURL + EndURL + TailURL

DateA = datetime.date(2001, 10,  1)
DateB = datetime.date(2001, 12, 31)
DateC = datetime.date(2016,  7,  1)
DateD = datetime.date(2016, 10,  1)

AppleURL = MakeYahooStockURL('AAPL', DateA, DateB)
print(AppleURL)
GoogleURL = MakeYahooStockURL('GOOG', DateC, DateD)
print(GoogleURL)

http://ichart.finance.yahoo.com/table.csv?&a=9&b=1&c=2001&d=11&e=31&f=2001&g=d&ignore=.csv&s=AAPL
http://ichart.finance.yahoo.com/table.csv?&a=6&b=1&c=2016&d=9&e=1&f=2016&g=d&ignore=.csv&s=GOOG


## Now let's download some data from that URL.

### This data is formatted very nicely and Pandas is smart enough to read it directly.
### For more complex data, we might use urllib.request.urlopen() from the standard Python Library.

In [13]:
AppleURL = MakeYahooStockURL('AAPL', datetime.date(2016, 3, 1), datetime.date(2016, 3, 7))  # 7 days of data
AAPL = pd.read_csv(AppleURL)

In [14]:
AAPL  # Type the variable to see it printed

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-03-07,102.389999,102.830002,100.959999,101.870003,35828900,100.707970
1,2016-03-04,102.370003,103.750000,101.370003,103.010002,46055100,101.834965
2,2016-03-03,100.580002,101.709999,100.449997,101.500000,36955700,100.342188
3,2016-03-02,100.510002,100.889999,99.639999,100.750000,33169600,99.600743
4,2016-03-01,97.650002,100.769997,97.419998,100.529999,50407100,99.383251


### If we were doing this a lot, we might write code that downloads the data to a local file or database, and read local data first if it exists.

# Other possible ways you might get data

### - Read data from a CSV file with csv.reader() or pd.read_csv()
### - Read data from an Excel spreadsheet with pd.read_excel()
### - Read data from a more complex URL with urllib.request.urlopen()
### - Read data from a database with SQLAlchemy
### - Read data from a JSON file with json.load()

# We're going to use stock data for the rest of the presentation

### Lets grab 10 years worth of stock ticker data for 10 stocks

In [15]:
StartDate = datetime.date(2006, 10,  1)
EndDate   = datetime.date(2016,  9, 30)

# Hack for development - get 0.5 years worth of data not 10
#StartDate = datetime.date(2013, 10,  1)
#EndDate   = datetime.date(2014,  3, 31)

Stocks = [
#    Symbol,  Name
    ('AAPL',  'Apple'    ),
    ('AMZN',  'Amazon'   ),
    ('DELL',  'Dell'     ),
    ('FB'  ,  'Facebook' ),
    ('GOOG',  'Google'   ),
    ('LNKD',  'LinkedIn' ),
    ('MSFT',  'Microsoft'),
    ('NFLX',  'Netflix'  ),
    ('TWTR',  'Twitter'  ),
    ('VMW' ,  'VMWare'   ),
]

HaveInternet = True
# Set this variable variable to True if I have Internet access, and all will be well.
# But if I don't have Internet access I don't want to mess up my presentation so set
# this variable to false and we'll load data we grabbed from the Internet earlier.

if HaveInternet:
    # Read data from the Internet
    StockData = pd.DataFrame()

    for Stock in Stocks:
        (Ticker, Name) = Stock
        
        # Read data for this Stock into Temp
        URL = MakeYahooStockURL(Ticker, StartDate, EndDate)
        Temp = pd.read_csv(URL, parse_dates=['Date'])
        Temp.set_index('Date', inplace=True)
        
        # Add the Adjusted Close price for this Stock to the StockData DataFrame
        StockData[Name] = Temp['Adj Close']
    
    # Save all of this data for later in case I don't have Internet access
    StockData.to_csv("Backup_in_case_I_dont_have_Internet.csv")
else:
    # Read data saved from earlier when I had Internet access
    StockData = pd.read_csv("Backup_in_case_I_dont_have_Internet.csv", index_col=['Date'], parse_dates=['Date'])

### DELL went private 2013-10-30 so it doesn't appear in the recent dates

In [16]:
StockData.head()  # DataFrame.head(n) shows the first n rows

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare
Date,,,,,,,,,,
2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,73.349998
2016-09-29,112.180000,829.049988,NaN,128.089996,775.010010,190.869995,57.400002,96.669998,23.010000,73.080002
2016-09-28,113.949997,828.719971,NaN,129.229996,781.559998,192.289993,58.029999,97.480003,22.959999,73.849998
2016-09-27,113.089996,816.109985,NaN,128.690002,783.010010,192.199997,57.950001,97.070000,23.719999,73.599998
2016-09-26,112.879997,799.159973,NaN,127.309998,774.210022,193.039993,56.900002,94.559998,23.370001,73.440002


### Several of these companies went public in the past 10 years so they don't appear in the earliest dates

In [17]:
StockData.tail()  # pandas.DataFrame.tail(n) shows the last n rows

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare
Date,,,,,,,,,,
2006-10-06,9.707185,32.590000,22.49090,NaN,210.040320,NaN,21.986586,3.444286,NaN,NaN
2006-10-05,9.786966,33.320000,22.52959,NaN,205.699647,NaN,22.026030,3.407143,NaN,NaN
2006-10-04,9.858901,32.759998,22.38449,NaN,207.642715,NaN,22.041809,3.314286,NaN,NaN
2006-10-03,9.688874,31.700001,21.95886,NaN,201.818531,NaN,21.592137,3.291429,NaN,NaN
2006-10-02,9.790890,30.870001,22.28776,NaN,200.519822,NaN,21.584248,3.258571,NaN,NaN


### Pandas is able to keep track of the indices of data, and align everything correctly

### Pandas marks missing data with NaNs

In [18]:
StockData.describe()  # pandas.DataFrame.describe() shows statistics on each numeric column

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare
count,2518.000000,2518.000000,1782.000000,1100.000000,2518.000000,1352.000000,2518.000000,2518.000000,730.000000,2300.000000
mean,55.866029,240.244889,16.060603,67.473027,380.648312,165.042411,29.725325,32.790608,34.891041,72.551104
std,35.469836,183.476451,5.141846,32.663947,169.772549,57.902648,10.838094,33.710346,13.717893,23.669950
min,9.577703,30.870001,7.729130,17.730000,128.591636,59.070000,12.464795,2.295714,14.010000,17.879999
25%,22.566719,84.295000,12.856090,31.345001,252.933752,107.629997,22.444147,5.590000,22.162499,56.542499
50%,50.695257,193.295006,14.230935,72.640000,305.135385,178.385002,24.913973,16.907857,36.414999,80.150002
75%,85.014889,322.132507,18.928630,93.492498,530.599898,213.420002,37.127508,51.304642,45.702500,90.195001
max,129.180748,837.309998,29.600930,131.050003,787.210022,270.760010,58.169998,130.929993,73.309998,124.830002


## I wonder much much data we are missing?

#### Pandas has functions to handle missing data/NaNs, but we need to use a Numpy function to count them.

#### Pandas is built upon Numpy arrays, so we can just use Numpy functions on Python DataFrames without any fuss.

In [19]:
# Count the NaNs
np.isnan(StockData).sum()

Apple           0
Amazon          0
Dell          736
Facebook     1418
Google          0
LinkedIn     1166
Microsoft       0
Netflix         0
Twitter      1788
VMWare        218
dtype: int64

In [20]:
# No, how many NaNs in the entire dataset?
np.isnan(StockData).sum().sum()

5326

In [21]:
# Now many data samples do we have for each ticker?
StockData.count()

Apple        2518
Amazon       2518
Dell         1782
Facebook     1100
Google       2518
LinkedIn     1352
Microsoft    2518
Netflix      2518
Twitter       730
VMWare       2300
dtype: int64

In [22]:
# How many data samples do we have all together?
StockData.count().sum()

19854

# Feature generation

### Lets focus on generating features from the date

In [23]:
# First we need to move the date from the index back to be a column
print("Our index columns are: {}".format(StockData.index.names))
print("Our data columns are: {}".format(StockData.columns))

print()
print("Resetting index...")
StockData.reset_index(inplace=True)
print()

print("Our index columns are: {}".format(StockData.index.names))
print("Our data columns are: {}".format(StockData.columns))

Our index columns are: ['Date']
Our data columns are: Index(['Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare'],
      dtype='object')

Resetting index...

Our index columns are: [None]
Our data columns are: Index(['Date', 'Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare'],
      dtype='object')


### Generate some date features using Pandas functions

#### Write code as if operating on a single row
#### Pandas then applies that code to every element in the series

In [24]:
StockData['Date-Weekday']     = StockData['Date'].dt.weekday  # day of the week with Monday=0, Sunday=6
StockData['Date-DayOfYear']   = StockData['Date'].dt.dayofyear
StockData['Date-DaysInMonth'] = StockData['Date'].dt.days_in_month
StockData['Date-DayOfMonth']  = StockData['Date'].dt.day
StockData['Date-Month']       = StockData['Date'].dt.month
StockData['Date-Quarter']     = StockData['Date'].dt.quarter
StockData['Date-Year']        = StockData['Date'].dt.year
StockData['Date-1st']         = StockData['Date'].dt.day == 1
StockData['Date-MonthStart']  = StockData['Date'].dt.is_month_start
StockData['Date-MonthEnd']    = StockData['Date'].dt.is_month_end

### When I create derivative features, I like to make the names show where the feature was derived from if possible

In [25]:
# The [] on a Pandas DataFrame picks out which columns (i.e. Pandas Series) we are referencing
StockData[['Date', 'Date-1st', 'Date-MonthStart', 'Date-MonthEnd']].head()

# An alternate way to do the same thing.
#Cols = ['Date', 'Date-1st', 'Date-MonthStart', 'Date-MonthEnd']
#StockData[Cols].head()

,Date,Date-1st,Date-MonthStart,Date-MonthEnd
0,2016-09-30,False,False,True
1,2016-09-29,False,False,False
2,2016-09-28,False,False,False
3,2016-09-27,False,False,False
4,2016-09-26,False,False,False


____
### Generate 1-hot variables for Monday to Friday

1-hot encoding is where you create a feature for every possible category of an input, and set it to 0 or 1 based on the input.

An example makes it easier to understand.

|DoW||Date-Mon|Date-Tue|Date-Wed|Date-Thu|Date-Fri|Date-Sat|Date-Sun|
|:-:||:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|Mon|| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
|Tue|| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
|Wed|| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
|Thu|| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
|Fri|| 0 | 0 | 0 | 0 | 1 | 0 | 0 |
|Sat|| 0 | 0 | 0 | 0 | 0 | 1 | 0 |
|Sun|| 0 | 0 | 0 | 0 | 0 | 0 | 1 |

It's hard to feed "Mon", "Tue", etc into a neural network, but 0s and 1s are no problem.

In [26]:
DayNames = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
for DoW, DayName in enumerate(DayNames):
    StockData['Date-{}'.format(DayName)] = (StockData['Date'].dt.weekday == DoW).astype(int)

In [27]:
StockData.head(20)

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Quarter,Date-Year,Date-1st,Date-MonthStart,Date-MonthEnd,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri
0,2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,...,3,2016,False,False,True,0,0,0,0,1
1,2016-09-29,112.180000,829.049988,NaN,128.089996,775.010010,190.869995,57.400002,96.669998,23.010000,...,3,2016,False,False,False,0,0,0,1,0
2,2016-09-28,113.949997,828.719971,NaN,129.229996,781.559998,192.289993,58.029999,97.480003,22.959999,...,3,2016,False,False,False,0,0,1,0,0
3,2016-09-27,113.089996,816.109985,NaN,128.690002,783.010010,192.199997,57.950001,97.070000,23.719999,...,3,2016,False,False,False,0,1,0,0,0
4,2016-09-26,112.879997,799.159973,NaN,127.309998,774.210022,193.039993,56.900002,94.559998,23.370001,...,3,2016,False,False,False,1,0,0,0,0
5,2016-09-23,112.709999,805.750000,NaN,127.959999,786.900024,192.789993,57.430000,95.940002,22.620001,...,3,2016,False,False,False,0,0,0,0,1
6,2016-09-22,114.620003,804.700012,NaN,130.080002,787.210022,192.479996,57.820000,95.830002,18.629999,...,3,2016,False,False,False,0,0,0,1,0
7,2016-09-21,113.550003,789.739990,NaN,129.940002,776.219971,192.399994,57.759998,94.879997,18.490000,...,3,2016,False,False,False,0,0,1,0,0
8,2016-09-20,113.570000,780.219971,NaN,128.639999,771.409973,192.500000,56.810001,98.250000,18.389999,...,3,2016,False,False,False,0,1,0,0,0
9,2016-09-19,113.580002,775.099976,NaN,128.649994,765.700012,192.199997,56.930000,98.059998,18.360001,...,3,2016,False,False,False,1,0,0,0,0


### Generate 1-hot version of Quarter

In [28]:
for Quarter in range(1,5):
    StockData['Date-Quarter{}'.format(Quarter)] = (StockData['Date'].dt.quarter == Quarter).astype(int)

In [29]:
StockData.columns

Index(['Date', 'Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare', 'Date-Weekday',
       'Date-DayOfYear', 'Date-DaysInMonth', 'Date-DayOfMonth', 'Date-Month',
       'Date-Quarter', 'Date-Year', 'Date-1st', 'Date-MonthStart',
       'Date-MonthEnd', 'Date-Mon', 'Date-Tue', 'Date-Wed', 'Date-Thu',
       'Date-Fri', 'Date-Quarter1', 'Date-Quarter2', 'Date-Quarter3',
       'Date-Quarter4'],
      dtype='object')

In [30]:
StockData.head(10)

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-MonthEnd,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Quarter1,Date-Quarter2,Date-Quarter3,Date-Quarter4
0,2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,...,True,0,0,0,0,1,0,0,1,0
1,2016-09-29,112.180000,829.049988,NaN,128.089996,775.010010,190.869995,57.400002,96.669998,23.010000,...,False,0,0,0,1,0,0,0,1,0
2,2016-09-28,113.949997,828.719971,NaN,129.229996,781.559998,192.289993,58.029999,97.480003,22.959999,...,False,0,0,1,0,0,0,0,1,0
3,2016-09-27,113.089996,816.109985,NaN,128.690002,783.010010,192.199997,57.950001,97.070000,23.719999,...,False,0,1,0,0,0,0,0,1,0
4,2016-09-26,112.879997,799.159973,NaN,127.309998,774.210022,193.039993,56.900002,94.559998,23.370001,...,False,1,0,0,0,0,0,0,1,0
5,2016-09-23,112.709999,805.750000,NaN,127.959999,786.900024,192.789993,57.430000,95.940002,22.620001,...,False,0,0,0,0,1,0,0,1,0
6,2016-09-22,114.620003,804.700012,NaN,130.080002,787.210022,192.479996,57.820000,95.830002,18.629999,...,False,0,0,0,1,0,0,0,1,0
7,2016-09-21,113.550003,789.739990,NaN,129.940002,776.219971,192.399994,57.759998,94.879997,18.490000,...,False,0,0,1,0,0,0,0,1,0
8,2016-09-20,113.570000,780.219971,NaN,128.639999,771.409973,192.500000,56.810001,98.250000,18.389999,...,False,0,1,0,0,0,0,0,1,0
9,2016-09-19,113.580002,775.099976,NaN,128.649994,765.700012,192.199997,56.930000,98.059998,18.360001,...,False,1,0,0,0,0,0,0,1,0


### We can deal with columns programmatically

In [31]:
# If we iterate through the Pandas DataFrame, we get the individual columns
for Column in StockData:
    print(Column)

# If you want to iterate through the rows in the DataFrame, use pd.DataFrame.iterrows()

Date
Apple
Amazon
Dell
Facebook
Google
LinkedIn
Microsoft
Netflix
Twitter
VMWare
Date-Weekday
Date-DayOfYear
Date-DaysInMonth
Date-DayOfMonth
Date-Month
Date-Quarter
Date-Year
Date-1st
Date-MonthStart
Date-MonthEnd
Date-Mon
Date-Tue
Date-Wed
Date-Thu
Date-Fri
Date-Quarter1
Date-Quarter2
Date-Quarter3
Date-Quarter4


In [32]:
# Rename columns
Columns = StockData.columns.values
Columns[-4:] = ['Date-Q{}'.format(N) for N in range(1,5)]
StockData.columns = Columns
print(StockData.columns)

Index(['Date', 'Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare', 'Date-Weekday',
       'Date-DayOfYear', 'Date-DaysInMonth', 'Date-DayOfMonth', 'Date-Month',
       'Date-Quarter', 'Date-Year', 'Date-1st', 'Date-MonthStart',
       'Date-MonthEnd', 'Date-Mon', 'Date-Tue', 'Date-Wed', 'Date-Thu',
       'Date-Fri', 'Date-Q1', 'Date-Q2', 'Date-Q3', 'Date-Q4'],
      dtype='object')


In [33]:
# Several different ways to delete columns
del(StockData['Date-1st'])
StockData = StockData.drop(['Date-DaysInMonth'], axis=1)
StockData.drop(['Date-MonthStart', 'Date-MonthEnd'], axis=1, inplace=True)

In [34]:
StockData.head()

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Year,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4
0,2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,...,2016,0,0,0,0,1,0,0,1,0
1,2016-09-29,112.180000,829.049988,NaN,128.089996,775.010010,190.869995,57.400002,96.669998,23.010000,...,2016,0,0,0,1,0,0,0,1,0
2,2016-09-28,113.949997,828.719971,NaN,129.229996,781.559998,192.289993,58.029999,97.480003,22.959999,...,2016,0,0,1,0,0,0,0,1,0
3,2016-09-27,113.089996,816.109985,NaN,128.690002,783.010010,192.199997,57.950001,97.070000,23.719999,...,2016,0,1,0,0,0,0,0,1,0
4,2016-09-26,112.879997,799.159973,NaN,127.309998,774.210022,193.039993,56.900002,94.559998,23.370001,...,2016,1,0,0,0,0,0,0,1,0


In [35]:
# Move the 'Date' field back into the index
StockData = StockData.set_index('Date')

StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Year,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,73.349998,...,2016,0,0,0,0,1,0,0,1,0
2016-09-29,112.180000,829.049988,NaN,128.089996,775.010010,190.869995,57.400002,96.669998,23.010000,73.080002,...,2016,0,0,0,1,0,0,0,1,0
2016-09-28,113.949997,828.719971,NaN,129.229996,781.559998,192.289993,58.029999,97.480003,22.959999,73.849998,...,2016,0,0,1,0,0,0,0,1,0
2016-09-27,113.089996,816.109985,NaN,128.690002,783.010010,192.199997,57.950001,97.070000,23.719999,73.599998,...,2016,0,1,0,0,0,0,0,1,0
2016-09-26,112.879997,799.159973,NaN,127.309998,774.210022,193.039993,56.900002,94.559998,23.370001,73.440002,...,2016,1,0,0,0,0,0,0,1,0


### All of the new features I created above were derived from a single column, but we can refer to multiple columns when generating features

In [36]:
StockNames = [StockName for _, StockName in Stocks]
print(StockNames)

StockData['MeanPrice'] = StockData[StockNames].mean(axis=1)  # axis=1 require to generate means per row
# Note that mean() is smart enough to deal with missing data

StockData['AMZN/AAPL'] = StockData['Amazon'] / StockData['Apple']

StockData[StockNames + ['MeanPrice', 'AMZN/AAPL']].head()

['Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn', 'Microsoft', 'Netflix', 'Twitter', 'VMWare']


,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,MeanPrice,AMZN/AAPL
Date,,,,,,,,,,,,
2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,73.349998,255.509997,7.406546
2016-09-29,112.180000,829.049988,NaN,128.089996,775.010010,190.869995,57.400002,96.669998,23.010000,73.080002,253.928888,7.390355
2016-09-28,113.949997,828.719971,NaN,129.229996,781.559998,192.289993,58.029999,97.480003,22.959999,73.849998,255.341106,7.272663
2016-09-27,113.089996,816.109985,NaN,128.690002,783.010010,192.199997,57.950001,97.070000,23.719999,73.599998,253.937776,7.216465
2016-09-26,112.879997,799.159973,NaN,127.309998,774.210022,193.039993,56.900002,94.559998,23.370001,73.440002,250.541110,7.079731


In [39]:
(StockData['Dell'] / StockData['Facebook']).tail()

Date
2006-10-06   NaN
2006-10-05   NaN
2006-10-04   NaN
2006-10-03   NaN
2006-10-02   NaN
dtype: float64

# Grouping and filtering

### Pandas is great at selecting and grouping data
#### - Use .loc() and .iloc() to select specific data
#### - Use .groupby() to split the data into groups

In [40]:
# Show us only the data where 'Date-Fri' is 1
StockData.loc[StockData['Date-Fri'] == 1].head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-30,113.050003,837.309998,NaN,128.270004,777.289978,191.119995,57.599998,98.550003,23.049999,73.349998,...,0,0,0,1,0,0,1,0,255.509997,7.406546
2016-09-23,112.709999,805.750000,NaN,127.959999,786.900024,192.789993,57.430000,95.940002,22.620001,73.379997,...,0,0,0,1,0,0,1,0,252.831113,7.148878
2016-09-16,114.919998,778.520020,NaN,129.070007,768.880005,192.399994,57.250000,99.480003,19.110001,72.440002,...,0,0,0,1,0,0,1,0,248.007781,6.774452
2016-09-09,103.129997,760.140015,NaN,127.099998,759.659973,192.740005,56.209999,96.500000,18.110001,73.080002,...,0,0,0,1,0,0,1,0,242.963332,7.370698
2016-09-02,107.730003,772.440002,NaN,126.510002,771.460022,192.800003,57.669998,97.379997,19.549999,73.779999,...,0,0,0,1,0,0,1,0,246.591114,7.170147


In [41]:
# Show us Twitter prices on Fridays in Q2
StockData.loc[(StockData['Date-Fri'] == 1) & (StockData['Date-Q2'] == 1), 'Twitter']
# Note we use the boolean '&' here, not the logical 'and' which will throw
# a complicated error message.

Date
2016-06-24    16.440001
2016-06-17    16.100000
2016-06-10    14.020000
2016-06-03    15.200000
2016-05-27    15.100000
2016-05-20    14.430000
2016-05-13    14.100000
2016-05-06    14.400000
2016-04-29    14.620000
2016-04-22    17.230000
2016-04-15    17.580000
2016-04-08    16.650000
2016-04-01    15.980000
2015-06-26    35.259998
2015-06-19    35.860001
2015-06-12    35.900002
2015-06-05    37.000000
2015-05-29    36.669998
2015-05-22    36.599998
2015-05-15    37.099998
2015-05-08    37.590000
2015-05-01    37.840000
2015-04-24    50.820000
2015-04-17    50.660000
2015-04-10    51.939999
2014-06-27    40.930000
2014-06-20    39.240002
2014-06-13    36.900002
2014-06-06    33.330002
2014-05-30    32.439999
                ...    
2009-05-08          NaN
2009-05-01          NaN
2009-04-24          NaN
2009-04-17          NaN
2009-04-03          NaN
2008-06-27          NaN
2008-06-20          NaN
2008-06-13          NaN
2008-06-06          NaN
2008-05-30          NaN
2008-05-23 

In [44]:
# Show us days when the Netflix price was more than 1.5x higher than the Microsoft price
StockA = 'Netflix'
StockB = 'Microsoft'
Multiplier = 2.3

DayCount = StockData.loc[(StockData[StockA] > Multiplier*StockData[StockB]), [StockA, StockB]].count()[0]
print("The {a} price was more than {m}x higher than the {b} price on {d} days".format(
        a=StockA, b=StockB, m=Multiplier, d=DayCount))

StockData.loc[(StockData[StockA] > Multiplier*StockData[StockB]), [StockA, StockB]]

The Netflix price was more than 2.3x higher than the Microsoft price on 73 days


,Netflix,Microsoft
Date,,
2015-12-09,124.199997,53.873132
2015-12-08,126.980003,54.666826
2015-12-04,130.929993,54.784409
2015-12-03,126.809998,53.108836
2015-12-02,128.929993,54.098501
2015-12-01,125.370003,54.108302
2015-11-30,123.330002,53.255814
2015-11-27,125.440002,52.844272
2015-11-25,124.160004,52.609102


### GroupBy splits the data into groups that we specify

In [45]:
StockData.columns

Index(['Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare', 'Date-Weekday',
       'Date-DayOfYear', 'Date-DayOfMonth', 'Date-Month', 'Date-Quarter',
       'Date-Year', 'Date-Mon', 'Date-Tue', 'Date-Wed', 'Date-Thu', 'Date-Fri',
       'Date-Q1', 'Date-Q2', 'Date-Q3', 'Date-Q4', 'MeanPrice', 'AMZN/AAPL'],
      dtype='object')

In [46]:
# GroupBy will split the data by whatever criteria you specify
for Quarter, Data in StockData.groupby('Date-Quarter'):
    print("Q{} mean prices".format(Quarter))
    print("There are {} rows in this group of data".format(len(Data)))
    print(Data[StockNames].mean())
    print()

Q1 mean prices
There are 611 rows in this group of data
Apple         53.003238
Amazon       221.776710
Dell          15.610098
Facebook      69.083539
Google       377.270058
LinkedIn     166.036328
Microsoft     28.976785
Netflix       30.600884
Twitter       39.481311
VMWare        69.893709
dtype: float64

Q2 mean prices
There are 633 rows in this group of data
Apple         56.924979
Amazon       243.459289
Dell          16.198949
Facebook      66.693697
Google       376.057472
LinkedIn     153.558098
Microsoft     29.992143
Netflix       33.924739
Twitter       31.365263
VMWare        73.526298
dtype: float64

Q3 mean prices
There are 638 rows in this group of data
Apple         60.509804
Amazon       272.813919
Dell          16.257144
Facebook      70.080000
Google       397.270648
LinkedIn     169.026058
Microsoft     30.787134
Netflix       38.036722
Twitter       31.341354
VMWare        73.886952
dtype: float64

Q4 mean prices
There are 636 rows in this group of data
Apple   

In [47]:
for (Year, Quarter), Data in StockData.groupby(['Date-Year','Date-Quarter']):
    print("{}-Q{} mean prices".format(Year, Quarter))
    print(Data[StockNames].mean())
    print()

2006-Q4 mean prices
Apple         10.833592
Amazon        37.814444
Dell          24.217389
Facebook            NaN
Google       232.783331
LinkedIn            NaN
Microsoft     22.925868
Netflix        3.833900
Twitter             NaN
VMWare              NaN
dtype: float64

2007-Q1 mean prices
Apple         11.577494
Amazon        38.559016
Dell          23.191720
Facebook            NaN
Google       234.624876
LinkedIn            NaN
Microsoft     23.069621
Netflix        3.254988
Twitter             NaN
VMWare              NaN
dtype: float64

2007-Q2 mean prices
Apple         14.165565
Amazon        60.800476
Dell          25.150667
Facebook            NaN
Google       243.182949
LinkedIn            NaN
Microsoft     23.783462
Netflix        3.140930
Twitter             NaN
VMWare              NaN
dtype: float64

2007-Q3 mean prices
Apple         17.737609
Amazon        79.486190
Dell          26.868853
Facebook            NaN
Google       263.041298
LinkedIn            NaN
Microsof

# Normalizing

### Neural networks often perform better if the inputs have a mean of 0.0, and standard-deviation of 1.0

In [48]:
# Create a data structure to record howe we modified variables to normalization them
DataDescription = {}

In [49]:
# Let's check that we actually normalized the data - before
StockData.describe()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL
count,2518.000000,2518.000000,1782.000000,1100.000000,2518.000000,1352.000000,2518.000000,2518.000000,730.000000,2300.000000,...,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,55.866029,240.244889,16.060603,67.473027,380.648312,165.042411,29.725325,32.790608,34.891041,72.551104,...,0.204925,0.205719,0.201350,0.199762,0.242653,0.251390,0.253376,0.252581,113.701722,4.217370
std,35.469836,183.476451,5.141846,32.663947,169.772549,57.902648,10.838094,33.710346,13.717893,23.669950,...,0.403727,0.404306,0.401089,0.399901,0.428772,0.433898,0.435031,0.434579,55.833579,1.033111
min,9.577703,30.870001,7.729130,17.730000,128.591636,59.070000,12.464795,2.295714,14.010000,17.879999,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.615220,2.274882
25%,22.566719,84.295000,12.856090,31.345001,252.933752,107.629997,22.444147,5.590000,22.162499,56.542499,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.418472,3.504079
50%,50.695257,193.295006,14.230935,72.640000,305.135385,178.385002,24.913973,16.907857,36.414999,80.150002,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.676766,4.107114
75%,85.014889,322.132507,18.928630,93.492498,530.599898,213.420002,37.127508,51.304642,45.702500,90.195001,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,161.635330,4.688175
max,129.180748,837.309998,29.600930,131.050003,787.210022,270.760010,58.169998,130.929993,73.309998,124.830002,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.509997,7.990029


In [50]:
for StockName in StockNames:
    # Calculate the mean and std-dev of each stock
    Mean = StockData[StockName].mean()
    StdDev = StockData[StockName].std()

    # Record the results in our data description
    DataDescription["{}-mean"  .format(StockName)] = Mean
    DataDescription["{}-stddev".format(StockName)] = StdDev
    
    # Renormalize the stock data
    print("{} mean = {:.2f}, std-dev = {:.2f}".format(StockName, Mean, StdDev))
    StockData[StockName] = (StockData[StockName] - Mean) / StdDev

Apple mean = 55.87, std-dev = 35.47
Amazon mean = 240.24, std-dev = 183.48
Dell mean = 16.06, std-dev = 5.14
Facebook mean = 67.47, std-dev = 32.66
Google mean = 380.65, std-dev = 169.77
LinkedIn mean = 165.04, std-dev = 57.90
Microsoft mean = 29.73, std-dev = 10.84
Netflix mean = 32.79, std-dev = 33.71
Twitter mean = 34.89, std-dev = 13.72
VMWare mean = 72.55, std-dev = 23.67


In [51]:
# Let's check that we actually normalized the data - after
StockData.describe()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL
count,2.518000e+03,2.518000e+03,1.782000e+03,1.100000e+03,2.518000e+03,1.352000e+03,2.518000e+03,2.518000e+03,7.300000e+02,2.300000e+03,...,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,-3.420968e-15,8.186903e-16,-2.406854e-15,-2.093679e-15,6.792731e-16,-3.974467e-17,-5.174442e-15,-1.357135e-15,4.091552e-15,6.765216e-15,...,0.204925,0.205719,0.201350,0.199762,0.242653,0.251390,0.253376,0.252581,113.701722,4.217370
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.403727,0.404306,0.401089,0.399901,0.428772,0.433898,0.435031,0.434579,55.833579,1.033111
min,-1.305005e+00,-1.141154e+00,-1.620327e+00,-1.522873e+00,-1.484673e+00,-1.830182e+00,-1.592580e+00,-9.046153e-01,-1.522176e+00,-2.309726e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.615220,2.274882
25%,-9.388064e-01,-8.499722e-01,-6.232223e-01,-1.106052e+00,-7.522686e-01,-9.915335e-01,-6.718136e-01,-8.068920e-01,-9.278788e-01,-6.763261e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.418472,3.504079
50%,-1.457794e-01,-2.558905e-01,-3.558387e-01,1.581858e-01,-4.447888e-01,2.304314e-01,-4.439298e-01,-4.711536e-01,1.110927e-01,3.210356e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.676766,4.107114
75%,8.217929e-01,4.463113e-01,5.577817e-01,7.965807e-01,8.832499e-01,8.354988e-01,6.829783e-01,5.492093e-01,7.881282e-01,7.454133e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,161.635330,4.688175
max,2.066960e+00,3.254178e+00,2.633359e+00,1.946396e+00,2.394744e+00,1.825782e+00,2.624509e+00,2.911254e+00,2.800646e+00,2.208661e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.509997,7.990029


In [52]:
DataDescription

{'Amazon-mean': 240.2448889864971,
 'Amazon-stddev': 183.47645123829923,
 'Apple-mean': 55.86602879388415,
 'Apple-stddev': 35.469836414296175,
 'Dell-mean': 16.06060268799103,
 'Dell-stddev': 5.14184565361458,
 'Facebook-mean': 67.4730272472728,
 'Facebook-stddev': 32.66394702353694,
 'Google-mean': 380.6483118594123,
 'Google-stddev': 169.77254943885603,
 'LinkedIn-mean': 165.04241129511834,
 'LinkedIn-stddev': 57.90264848769859,
 'Microsoft-mean': 29.72532536934081,
 'Microsoft-stddev': 10.838093600863495,
 'Netflix-mean': 32.79060815170775,
 'Netflix-stddev': 33.710345937050306,
 'Twitter-mean': 34.89104106849309,
 'Twitter-stddev': 13.717892755135985,
 'VMWare-mean': 72.55110437608681,
 'VMWare-stddev': 23.669950454845722}

### It's a good idea to save normalizion data in file
If you've trained a neural network model on normalized data, you MUST use the same normalization parameters when you're running the model

#### We'll save this data in a JSON file

In [53]:
# Save the data description in a file
DataDescriptionFileName = "DataDescription.json"
with open(DataDescriptionFileName, 'w') as f:
    json.dump(DataDescription, f, indent=4)
print("Saved data description to {}".format(DataDescriptionFileName))

Saved data description to DataDescription.json


### We can execute shell commands from within an IPython notebook by prepending the command with "!"

In [54]:
!ls -l
!head DataDescription.json

total 2576
-rw-r--r--  1 dannymulligan  staff  262851 Oct 18 19:35 Backup_in_case_I_dont_have_Internet.csv
-rw-r--r--  1 dannymulligan  staff     800 Oct 18 20:00 DataDescription.json
-rw-r--r--  1 dannymulligan  staff  180739 Oct 18 19:59 Data_Munging.ipynb
-rw-r--r--  1 dannymulligan  staff  617975 Oct 18 15:05 StockData.csv
-rw-r--r--  1 dannymulligan  staff  245419 Oct 18 15:05 StockData.csv.gz
{
    "Apple-mean": 55.86602879388415,
    "Google-stddev": 169.77254943885603,
    "Amazon-mean": 240.2448889864971,
    "Twitter-mean": 34.89104106849309,
    "LinkedIn-stddev": 57.90264848769859,
    "Google-mean": 380.6483118594123,
    "Microsoft-mean": 29.72532536934081,
    "Microsoft-stddev": 10.838093600863495,
    "Dell-mean": 16.06060268799103,


# Other transformations

### This presentation is already too long, so I can't cover these, but other things you might do include...

#### - Stacking and unstacking data
Moving data from rows to columns or visa-versa
#### - Filling in missing data by various means
#### - Calculating rolling statistics
#### - Resampling data
Say from daily to weekly or monthly
#### - Sorting, dealing with duplicates, looking for unique values
#### - Lots more

# Partitioning data into train/test sets

### It's common to present training data to a neural network in mini-batches
### It is also common to hold back a portion of the data to use as test data

In [55]:
# Mark everything as 'train' data, we'll overwrite some of it as 'test' data later
StockData['Set'] = 'train'

### Also, let's assume we can't use the initial data samples in our training or test sets, because we don't have enough previous history to do something useful with it.

In [56]:
# We want to find the first date in the table 
# We do this by...
#    Grabbing the index which contains the date with "StockData.index"
#    Sorting it with "np.sort()"
#    Grabbing the first (i.e. earliest) date with "[0]"
#    Converting from Numpy timestamps to datetime.datetime with "pd.to_datetime()"
#    Converting from datetime.datetime to datetime.date with ".date()"
# Ugh, I'm not going to sugar coat this one, this code is UGLY
FirstDate = pd.to_datetime(np.sort(StockData.index)[0]).date()

DaysHistoryRequired = 90
StartDate = FirstDate + datetime.timedelta(days=DaysHistoryRequired)
print("Dates prior to {} will be marked as history".format(StartDate))

if StartDate == datetime.date(1970,4,1):
    print("Warning, you are getting a bad start date because you're running this code while 'Date' is not in the Index")

Dates prior to 2006-12-31 will be marked as history


In [57]:
# We need 'Date' as a column for a bit
StockData.reset_index(inplace=True)

In [58]:
StockData.loc[StockData.Date < StartDate, 'Set'] = 'history'

print("We now have {:,} rows marked as 'history'".format(len(StockData.loc[StockData.Set == 'history'])))
print("We now have {:,} rows marked as 'train'".format(len(StockData.loc[StockData.Set == 'train'])))

We now have 63 rows marked as 'history'
We now have 2,455 rows marked as 'train'


In [59]:
# Move 'Date' back to the index
StockData.set_index(['Date'], inplace=True)

In [60]:
StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-30,1.612186,3.254178,NaN,1.861287,2.336312,0.450369,2.571917,1.950718,-0.863182,0.033751,...,0,0,1,0,0,1,0,255.509997,7.406546,train
2016-09-29,1.587658,3.209159,NaN,1.855776,2.322883,0.446052,2.553464,1.894949,-0.866098,0.022345,...,0,1,0,0,0,1,0,253.928888,7.390355,train
2016-09-28,1.637560,3.207360,NaN,1.890677,2.361464,0.470576,2.611592,1.918977,-0.869743,0.054875,...,1,0,0,0,0,1,0,255.341106,7.272663,train
2016-09-27,1.613314,3.138632,NaN,1.874145,2.370004,0.469021,2.604210,1.906815,-0.814341,0.044313,...,0,0,0,0,0,1,0,253.937776,7.216465,train
2016-09-26,1.607393,3.046250,NaN,1.831897,2.318170,0.483529,2.507330,1.832357,-0.839855,0.037554,...,0,0,0,0,0,1,0,250.541110,7.079731,train


In [61]:
StockData.tail()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2006-10-06,-1.301355,-1.131780,1.250582,NaN,-1.004921,NaN,-0.714031,-0.870543,NaN,NaN,...,0,0,1,0,0,0,1,50.043213,3.357307,history
2006-10-05,-1.299106,-1.127801,1.258106,NaN,-1.030489,NaN,-0.710392,-0.871645,NaN,NaN,...,0,1,0,0,0,0,1,49.461563,3.404528,history
2006-10-04,-1.297078,-1.130853,1.229887,NaN,-1.019043,NaN,-0.708936,-0.874400,NaN,NaN,...,1,0,0,0,0,0,1,49.667033,3.322885,history
2006-10-03,-1.301871,-1.136630,1.147109,NaN,-1.053349,NaN,-0.750426,-0.875078,NaN,NaN,...,0,0,0,0,0,0,1,48.341639,3.271794,history
2006-10-02,-1.298995,-1.141154,1.211074,NaN,-1.060999,NaN,-0.751154,-0.876053,NaN,NaN,...,0,0,0,0,0,0,1,48.051882,3.152931,history


### We're going to randomly mark 20% of the 'train' data as 'test'

In [62]:
TestRatio = 0.2

In [63]:
# Switch to an integer index by moving 'Date' back to being a column
StockData.reset_index(inplace=True)
StockData.index.values

array([   0,    1,    2, ..., 2515, 2516, 2517])

In [64]:
SampleIndex = StockData.loc[StockData.Set == 'train'].index.values.copy()  # Make a copy to avoid modifying StockData.index
NumRows = len(SampleIndex)
NumTest = int(TestRatio * NumRows)
NumTrain = NumRows - NumTest
print("We have {:,} rows, we will use {:,} of them for training, {:,} for testing.".format(NumRows, NumTrain, NumTest))

We have 2,455 rows, we will use 1,964 of them for training, 491 for testing.


In [65]:
np.random.shuffle(SampleIndex)

In [66]:
TrainIndex = SampleIndex[:NumTrain]

In [67]:
TestIndex = SampleIndex[NumTrain:]
StockData.loc[TestIndex, 'Set'] = 'test'

In [68]:
StockData.head(10)

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
0,2016-09-30,1.612186,3.254178,NaN,1.861287,2.336312,0.450369,2.571917,1.950718,-0.863182,...,0,0,1,0,0,1,0,255.509997,7.406546,train
1,2016-09-29,1.587658,3.209159,NaN,1.855776,2.322883,0.446052,2.553464,1.894949,-0.866098,...,0,1,0,0,0,1,0,253.928888,7.390355,train
2,2016-09-28,1.637560,3.207360,NaN,1.890677,2.361464,0.470576,2.611592,1.918977,-0.869743,...,1,0,0,0,0,1,0,255.341106,7.272663,train
3,2016-09-27,1.613314,3.138632,NaN,1.874145,2.370004,0.469021,2.604210,1.906815,-0.814341,...,0,0,0,0,0,1,0,253.937776,7.216465,test
4,2016-09-26,1.607393,3.046250,NaN,1.831897,2.318170,0.483529,2.507330,1.832357,-0.839855,...,0,0,0,0,0,1,0,250.541110,7.079731,train
5,2016-09-23,1.602600,3.082167,NaN,1.851796,2.392918,0.479211,2.556231,1.873294,-0.894528,...,0,0,1,0,0,1,0,252.831113,7.148878,test
6,2016-09-22,1.656449,3.076445,NaN,1.916700,2.394744,0.473857,2.592216,1.870031,-1.185389,...,0,1,0,0,0,1,0,252.797782,7.020590,train
7,2016-09-21,1.626282,2.994908,NaN,1.912414,2.330010,0.472475,2.586679,1.841850,-1.195595,...,1,0,0,0,0,1,0,249.599995,6.954998,train
8,2016-09-20,1.626846,2.943021,NaN,1.872614,2.301678,0.474203,2.499026,1.941819,-1.202885,...,0,0,0,0,0,1,0,248.093327,6.869948,train
9,2016-09-19,1.627128,2.915116,NaN,1.872920,2.268045,0.469021,2.510098,1.936183,-1.205071,...,0,0,0,0,0,1,0,246.806664,6.824265,train


### Create a function to supply training samples in batches

In [69]:
BatchSize = 10
TrainEpoch = 1
TrainBatchStart = 0
np.random.shuffle(TrainIndex)

def TrainingBatch():
    global TrainBatchStart
    global TrainEpoch
    
    if (TrainBatchStart + BatchSize) > NumTrain:
        # Done with this Epoch, start a new one
        TrainEpoch += 1
        np.random.shuffle(TrainIndex)
        TrainBatchStart = 0
        
    BatchIndex = TrainIndex[TrainBatchStart:TrainBatchStart + BatchSize]
    TrainBatchStart += BatchSize
    return StockData.loc[BatchIndex]

In [103]:
TrainingBatch()

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
2425,2007-02-14,-1.260499,-1.090630,1.372868,NaN,-0.871258,NaN,-0.587902,-0.876943,NaN,...,1,0,0,1,0,0,0,55.621806,3.597957,train
726,2013-11-12,0.408048,0.595636,NaN,-0.638717,0.734731,0.763999,0.433786,0.441559,0.510936,...,0,0,0,0,0,0,1,153.978597,4.969190,train
1731,2009-11-13,-0.821153,-0.584679,-0.226262,NaN,-0.559034,NaN,-0.466338,-0.721289,NaN,...,0,0,1,0,0,0,1,76.374808,4.972722,train
2179,2008-02-06,-1.125174,-0.936114,0.464185,NaN,-0.765987,NaN,-0.631319,-0.862449,NaN,...,1,0,0,1,0,0,0,63.088381,4.292348,train
1902,2009-03-13,-1.221303,-0.935351,-1.360705,NaN,-1.287606,NaN,-1.478710,-0.805875,NaN,...,0,0,1,1,0,0,0,42.040305,5.469998,train
1407,2011-03-01,-0.287005,-0.385907,-0.225791,NaN,-0.474564,NaN,-0.669816,-0.105539,NaN,...,0,0,0,1,0,0,0,94.782104,3.708793,train
2173,2008-02-14,-1.105041,-0.896272,0.603403,NaN,-0.676132,NaN,-0.632800,-0.859356,NaN,...,0,1,0,1,0,0,0,66.611661,4.546979,train
689,2014-01-07,0.484434,0.859975,NaN,-0.292464,1.108623,0.770217,0.376492,0.466011,1.936810,...,0,0,0,1,0,0,0,171.775186,5.448823,train
1765,2009-09-28,-0.888631,-0.806833,-0.175467,NaN,-0.775343,NaN,-0.758274,-0.777906,NaN,...,0,0,0,0,0,1,0,64.080778,3.787413,train
1559,2010-07-23,-0.616542,-0.661528,-0.581832,NaN,-0.800263,NaN,-0.732704,-0.516307,NaN,...,0,0,1,0,0,1,0,75.205979,3.496446,train


In [104]:
TrainEpoch

1

# Saving and loading data

In [105]:
!rm StockData.csv StockData.csv.gz
StockDataFile = 'StockData.csv'

StockData.to_csv(StockDataFile, index=False)

In [106]:
!ls -l StockData*

-rw-r--r--  1 dannymulligan  staff  617975 Oct 18 20:07 StockData.csv


#### This file isn't big enough for it to matter, but sometimes we can save a lot of time & space by reading & writing compressed data

In [107]:
StockData.to_csv(StockDataFile + '.gz', compression='gzip', index=False)

In [108]:
!ls -l StockData*

-rw-r--r--  1 dannymulligan  staff  617975 Oct 18 20:07 StockData.csv
-rw-r--r--  1 dannymulligan  staff  245396 Oct 18 20:07 StockData.csv.gz


In [109]:
# Delete the datastructure and read it back in from a file
del(StockData)

In [110]:
%timeit StockData = pd.read_csv(StockDataFile, index_col=['Date'], parse_dates=['Date'])

100 loops, best of 3: 16 ms per loop


In [111]:
%timeit StockData = pd.read_csv(StockDataFile + '.gz', index_col=['Date'], parse_dates=['Date'])

10 loops, best of 3: 23.8 ms per loop


#### In this case, reading the compressed file takes 23.4 ms vs 14.9 ms for the uncompressed file, but for large sparse data, the compressed file will sometimes be much faster

In [112]:
StockData = pd.read_csv(StockDataFile + '.gz', index_col=['Date'], parse_dates=['Date'])
StockData.head(10)

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-30,1.612186,3.254178,NaN,1.861287,2.336312,0.450369,2.571917,1.950718,-0.863182,0.033751,...,0,0,1,0,0,1,0,255.509997,7.406546,train
2016-09-29,1.587658,3.209159,NaN,1.855776,2.322883,0.446052,2.553464,1.894949,-0.866098,0.022345,...,0,1,0,0,0,1,0,253.928888,7.390355,train
2016-09-28,1.637560,3.207360,NaN,1.890677,2.361464,0.470576,2.611592,1.918977,-0.869743,0.054875,...,1,0,0,0,0,1,0,255.341106,7.272663,train
2016-09-27,1.613314,3.138632,NaN,1.874145,2.370004,0.469021,2.604210,1.906815,-0.814341,0.044313,...,0,0,0,0,0,1,0,253.937776,7.216465,test
2016-09-26,1.607393,3.046250,NaN,1.831897,2.318170,0.483529,2.507330,1.832357,-0.839855,0.037554,...,0,0,0,0,0,1,0,250.541110,7.079731,train
2016-09-23,1.602600,3.082167,NaN,1.851796,2.392918,0.479211,2.556231,1.873294,-0.894528,0.035019,...,0,0,1,0,0,1,0,252.831113,7.148878,test
2016-09-22,1.656449,3.076445,NaN,1.916700,2.394744,0.473857,2.592216,1.870031,-1.185389,0.053185,...,0,1,0,0,0,1,0,252.797782,7.020590,train
2016-09-21,1.626282,2.994908,NaN,1.912414,2.330010,0.472475,2.586679,1.841850,-1.195595,0.036709,...,1,0,0,0,0,1,0,249.599995,6.954998,train
2016-09-20,1.626846,2.943021,NaN,1.872614,2.301678,0.474203,2.499026,1.941819,-1.202885,0.021077,...,0,0,0,0,0,1,0,248.093327,6.869948,train


In [113]:
StockData.describe()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL
count,2.518000e+03,2.518000e+03,1.782000e+03,1.100000e+03,2.518000e+03,1.352000e+03,2.518000e+03,2.518000e+03,7.300000e+02,2.300000e+03,...,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,-3.438605e-15,9.088132e-16,-2.350907e-15,-2.067841e-15,6.678975e-16,-5.419728e-17,-5.264741e-15,-1.357135e-15,4.091704e-15,6.765603e-15,...,0.204925,0.205719,0.201350,0.199762,0.242653,0.251390,0.253376,0.252581,113.701722,4.217370
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.403727,0.404306,0.401089,0.399901,0.428772,0.433898,0.435031,0.434579,55.833579,1.033111
min,-1.305005e+00,-1.141154e+00,-1.620327e+00,-1.522873e+00,-1.484673e+00,-1.830182e+00,-1.592580e+00,-9.046153e-01,-1.522176e+00,-2.309726e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.615220,2.274882
25%,-9.388064e-01,-8.499722e-01,-6.232223e-01,-1.106052e+00,-7.522686e-01,-9.915335e-01,-6.718136e-01,-8.068920e-01,-9.278788e-01,-6.763261e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.418472,3.504079
50%,-1.457794e-01,-2.558905e-01,-3.558387e-01,1.581858e-01,-4.447888e-01,2.304314e-01,-4.439298e-01,-4.711536e-01,1.110927e-01,3.210356e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.676766,4.107114
75%,8.217929e-01,4.463113e-01,5.577817e-01,7.965807e-01,8.832499e-01,8.354988e-01,6.829783e-01,5.492093e-01,7.881282e-01,7.454133e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,161.635330,4.688175
max,2.066960e+00,3.254178e+00,2.633359e+00,1.946396e+00,2.394744e+00,1.825782e+00,2.624509e+00,2.911254e+00,2.800646e+00,2.208661e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.509997,7.990029


In [114]:
StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-30,1.612186,3.254178,NaN,1.861287,2.336312,0.450369,2.571917,1.950718,-0.863182,0.033751,...,0,0,1,0,0,1,0,255.509997,7.406546,train
2016-09-29,1.587658,3.209159,NaN,1.855776,2.322883,0.446052,2.553464,1.894949,-0.866098,0.022345,...,0,1,0,0,0,1,0,253.928888,7.390355,train
2016-09-28,1.637560,3.207360,NaN,1.890677,2.361464,0.470576,2.611592,1.918977,-0.869743,0.054875,...,1,0,0,0,0,1,0,255.341106,7.272663,train
2016-09-27,1.613314,3.138632,NaN,1.874145,2.370004,0.469021,2.604210,1.906815,-0.814341,0.044313,...,0,0,0,0,0,1,0,253.937776,7.216465,test
2016-09-26,1.607393,3.046250,NaN,1.831897,2.318170,0.483529,2.507330,1.832357,-0.839855,0.037554,...,0,0,0,0,0,1,0,250.541110,7.079731,train


____
# Bonus Material

## This lecture is really good
### https://www.youtube.com/watch?v=wofXCQXq1pg
### I found it via...
### https://www.reddit.com/r/MachineLearning/comments/54bps/yann_lecun_deep_learning_and_the_future_of_ai/

### This is one of the papers referenced in the lecture:
### http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
### It explains *why* we need to do some of the above data manipulations